# Import Libraries

In [73]:
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy, copy
import os
from multiprocessing import Pool

# Random seed

In [74]:
RANDOM_SEED = 10
np.random.seed(RANDOM_SEED)

# Functions to transform the data

In [75]:
# Function to transfer from dBW to W (power)
def db2pow(db: float) -> float:
    return 10**(db/10)

# Function to transfer from W to dBW (power)
def pow2db(pow: float) -> float:
    return 10*np.log10(pow)

# Hermitian transpose of a matrix
def HermTranspose(x: np.ndarray) -> np.ndarray:
    return x.conj().T

def chanGen(zeta: float, d: float, dim1: int, dim2: int) -> np.ndarray:
    """Function to generate Rayleigh fading channel coefficients

    Args:
        zeta: ξ is the path loss exponent
        d: the distance between the transmitter and the receiver
        dim1: the number of rows in the channel matrix
        dim2: the number of columns in the channel matrix
    """
    pl_ref: float = -30                                    # pathloss (dBW) at reference distance
    pl: float = db2pow(pl_ref - 10*zeta*np.log10(d))       # pathloss model at distance d
    y: np.ndarray = np.sqrt(0.5*pl)*(np.random.randn(dim1,dim2)\
        + 1j*np.random.randn(dim1,dim2))            # Rayleigh distribution
    return y

# Parameters

In [76]:
sigma = db2pow(-75)                                                                 # noise power
N = 10                                                                              # number of transmit antennas
zetaAI = 2.2                                                                        # Path loss exponent of the channel between the Alice and the RIS
zetaIB = 2.5                                                                        # Path loss exponent of the channel between the legitimate receivers and the RIS
zetaIE = 2.5                                                                        # Path loss exponent of the channel between the eavesdroppers and the RIS
zetaAB = 3.5                                                                        # Path loss exponent of the channel between the Alice and the legitimate receivers
zetaAE = 3.5                                                                        # Path loss exponent of the channel between the Alice and the eavesdroppers

pmax = 1000  # Maximum transmit power of Alice


In [77]:
def generateChannel(Nris, number_of_users, number_of_eavesdroppers):
    dAI = 50                                                                            # distance between Alice and the RIS
    dv = 2                                                                              # Vertical distance between the Alice and the Eve and Bob
    dABh = np.random.uniform(5, 10, size=number_of_users)                               # Horizontal distance between Alice and the legitimate receivers
    dAEh = np.random.uniform(50, 150, size=number_of_eavesdroppers)                     # Horizontal distance between Alice and the eavesdroppers
    dAB = [np.sqrt(dABh[i]**2 + dv**2) for i in range(number_of_users)]                 # Distance between Alice and the legitimate receivers
    dAE = [np.sqrt(dAEh[i]**2 + dv**2) for i in range(number_of_eavesdroppers)]         # Distance between Alice and the eavesdroppers
    dIB = [np.sqrt((dABh[i]-dAI)**2 + dv**2) for i in range(number_of_users)]           # Distance between the legitimate receivers and the RIS
    dIE = [np.sqrt((dAEh[i]-dAI)**2 + dv**2) for i in range(number_of_eavesdroppers)]   # Distance between the eavesdroppers and the RIS

    normFact: float = 1/np.sqrt(sigma)
    Hai = chanGen(zetaAI, dAI, Nris, N)                                                         # Alice to RIS channel
    hib = [normFact*chanGen(zetaIB, dIB[i], 1, Nris) for i in range(number_of_users)]           # Channel between the RIS and the legitimate receivers
    hie = [normFact*chanGen(zetaIE, dIE[i], 1, Nris) for i in range(number_of_eavesdroppers)]   # Channel between the RIS and the eavesdroppers
    hab = [normFact*chanGen(zetaAB, dAB[i], 1, N) for i in range(number_of_users)]              # Channel between Alice and the legitimate receivers
    hae = [normFact*chanGen(zetaAE, dAE[i], 1, N) for i in range(number_of_eavesdroppers)]      # Channel between Alice and the eavesdroppers
    return Hai, hib, hie, hab, hae

    #Hai: Channel between Alice and RIS: Nris x N  
    #hib: Channel between RIS and users: List of length number_of_users, elements are 1 x Nris
    #hab: Channel between Alice and users: List of length number_of_users, elements are 1 x N
    #hie: Channel between RIS and eavesdroppers: List of length number_of_eavesdroppers, elements are 1 x Nris
    #hae: Channel between Alice and eavesdroppers: List of length number_of_eavesdroppers, elements are 1 x N

# Functions for data generation

In [78]:
Nris = 64  # number of RIS elements

if os.path.exists('data') == False:
    os.mkdir('data')

# Generate general data of 10 users and 3 eavesdroppers for algorithms convergence
def gen_general_data() -> None:
    os.chdir('data')
    if os.path.exists('general') == False:
        os.mkdir('general')
        os.chdir('general')
        Hai, hib, hie, hab, hae = generateChannel(Nris, number_of_users = 10, number_of_eavesdroppers = 3)
        np.save('Hai.npy', Hai)
        np.save('hib.npy', hib)
        np.save('hie.npy', hie)
        np.save('hab.npy', hab)
        np.save('hae.npy', hae)
        os.chdir('../')
    os.chdir('../')

# Generate fixed number of eavesdroppers for different number of users
def gen_fixed_eaves_data() -> None:
    os.chdir('data')
    if os.path.exists('fixed_eaves') == False:
        os.mkdir('fixed_eaves')
    os.chdir('fixed_eaves')
    number_of_eavesdroppers = 3
        
    for number_of_users in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
        dirname = 'data_' + str(number_of_users) + 'users_' + str(number_of_eavesdroppers) + 'eaves'
        if os.path.exists(dirname) == False:
            os.mkdir(dirname)
        os.chdir(dirname)
        seed_number = RANDOM_SEED
        if os.path.exists(str(seed_number) + '_set') == False:
            os.mkdir(str(seed_number) + '_set')
            os.chdir(str(seed_number) + '_set')
            Hai, hib, hie, hab, hae = generateChannel(Nris, number_of_users, number_of_eavesdroppers)
            np.save('Hai.npy', Hai)
            np.save('hib.npy', hib)
            np.save('hie.npy', hie)
            np.save('hab.npy', hab)
            np.save('hae.npy', hae)
            os.chdir('../')
        os.chdir('../')
    os.chdir('../')
    os.chdir('../')

# Generate fixed number of users for different number of eavesdroppers
def gen_fixed_users_data() -> None:
    os.chdir('data')
    if os.path.exists('fixed_users') == False:
        os.mkdir('fixed_users')
    os.chdir('fixed_users')

    number_of_users = 6
    for number_of_eavesdroppers in [1, 2, 3, 4, 5]:
        dirname = 'data_' + str(number_of_users) + 'users_' + str(number_of_eavesdroppers) + 'eaves'
        if os.path.exists(dirname) == False:
            os.mkdir(dirname)
        os.chdir(dirname)
        seed_number = RANDOM_SEED
        if os.path.exists(str(seed_number) + '_set') == False:
            os.mkdir(str(seed_number) + '_set')
            os.chdir(str(seed_number) + '_set')
            Hai, hib, hie, hab, hae = generateChannel(Nris, number_of_users, number_of_eavesdroppers)
            np.save('Hai.npy', Hai)
            np.save('hib.npy', hib)
            np.save('hie.npy', hie)
            np.save('hab.npy', hab)
            np.save('hae.npy', hae)
            os.chdir('../')
        os.chdir('../')
    os.chdir('../')
    os.chdir('../')



# Generate data

In [79]:
gen_general_data()
gen_fixed_eaves_data()
gen_fixed_users_data()

In [80]:
os.getcwd()

'/Users/leducanhvu/Downloads/IoT_Lab/Security/RIS-assisted-secrecy-rate'